In [ ]:
# ! pip install pymysql


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import datetime
import pandas as pd
import numpy as np
import pymysql

In [13]:
# 공휴일 데이터 불러오기
df_holiday = pd.read_csv("D:/humanda/project/1st_miniproject/Mook/hd_21_24.csv")

In [14]:
# 데이터 확인
print(df_holiday.head(10))
print(df_holiday.info())

           일시 dateName
0  2021-01-01     1월1일
1  2021-02-11       설날
2  2021-02-12       설날
3  2021-02-13       설날
4  2021-03-01      삼일절
5  2021-05-05     어린이날
6  2021-05-19   부처님오신날
7  2021-06-06      현충일
8  2021-08-15      광복절
9  2021-09-21       추석
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   일시        70 non-null     object
 1   dateName  70 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB
None


In [15]:
# 공휴일 데이터의 일시를 날짜 데이터로 변경
df_holiday['일시'] = pd.to_datetime(df_holiday['일시'])

In [16]:
# 날짜 데이터 생성(2022-01-01 ~ 2024-10-31 )
date_range = pd.date_range(start='2022-01-01', end='2024-10-31', freq='D')

In [17]:
# 데이터프레임으로 저장
df_date = pd.DataFrame(date_range, columns=['date'])

In [18]:
# 날짜 데이터에 공휴일이거나 주말이면 1 아니면 0이라는 컬럼 추가
df_date['holiday_or_weekend'] = df_date['date'].apply(
    lambda x: 1 if (x.dayofweek >= 5 or x in df_holiday['일시'].values) else 0
)

In [19]:
# 확인
print(df_date.head(10))
print(df_date.info())

        date  holiday_or_weekend
0 2022-01-01                   1
1 2022-01-02                   1
2 2022-01-03                   0
3 2022-01-04                   0
4 2022-01-05                   0
5 2022-01-06                   0
6 2022-01-07                   0
7 2022-01-08                   1
8 2022-01-09                   1
9 2022-01-10                   0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                1035 non-null   datetime64[ns]
 1   holiday_or_weekend  1035 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 16.3 KB
None


In [20]:
# db 연결
conn = pymysql.connect(host="localhost", port=3306, db="project", 
                       user="humanda", password="humanda")

OperationalError: (2003, "Can't connect to MySQL server on 'localhost' ([WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다)")

In [ ]:
# 명령 도구 준비
cursor = conn.cursor()

In [ ]:
# 명령 실행 (테이블 생성)
sql = """create table dates
         (
            ID INT AUTO_INCREMENT PRIMARY KEY,         
            date date,
            holiday_or_weekend int
         )"""
cursor.execute(sql)

In [ ]:
# 명령 실행 (데이터 삽입)
sql2 = "insert into dates(date, holiday_or_weekend) values (%s, %s)"
for index, row in df_date.iterrows():
    cursor.execute(sql2, tuple(row))

In [ ]:
# 원본 테이블에 적용
conn.commit()

In [ ]:
# 명령 실행 (조회)
sql3 = "select * from dates"
cursor.execute(sql3)

In [ ]:
# 조회 결과 읽기
data = cursor.fetchall()

In [ ]:
# 표시
for row in data:
    print(row)

In [ ]:
# db에서 불러와 데이터프레임으로 저장
df_date = pd.read_sql_query(sql3, conn)

In [ ]:
df_date

In [ ]:
cursor.close()
conn.close()

In [ ]:
df_date.to_csv('df_date.csv', index=False)